В этом ноутбуке мы будем решать задачу классификации - "Ирисы Фишера".




---



Ирисы Фишера состоят из данных о 150 экземплярах ириса, по 50 экземпляров из трёх видов

-Ирис щетинистый (Iris setosa),

-Ирис виргинский (Iris virginica)

-Ирис разноцветный (Iris versicolor).



---


Для каждого экземпляра измерялись четыре характеристики (в сантиметрах):

Длина наружной доли околоцветника (англ. sepal length);



Ширина наружной доли околоцветника (англ. sepal width);

Длина внутренней доли околоцветника (англ. petal length);

Ширина внутренней доли околоцветника (англ. petal width).

На основании этого набора данных требуется построить правило классификации, определяющее вид растения по данным измерений. Это задача многоклассовой классификации, так как имеется три класса — три вида ириса.

Один из классов (Iris setosa) линейно-разделим от двух остальных.

In [13]:
#импортируем все необходимые библиотеки

import torch #для создания и обучения нейросети

import pandas as pd #для анализа данных

import matplotlib #для визуализации данных и хода обучения (даёт возможность рисовать графики)

from sklearn import datasets #импортируем датасет Ирисов


In [14]:
#давайте сперва посмотрим на датасет
data = datasets.load_iris()
train_data = torch.tensor(data['data'], dtype=torch.float32)
target_data = torch.tensor(data['target'])
train_data.shape

torch.Size([150, 4])

In [15]:
#Вообще, в PyTorch есть реализация линейного (полносвязного) слоя,
#но ради досконального понимания устройства нейросетей лучше разок
#написать его самим.

class Linear_layer(torch.nn.Module):
  def __init__(self, in_features, out_features):
      super().__init__()
      self.weight = torch.nn.Parameter(torch.randn(in_features))
      self.bias = torch.nn.Parameter(torch.zeros(out_features))
  def forward(self, input):
      return (input @ self.weight) + self.bias


Итого мы имеем: задача многоклассовой классификации (3 класса), у каждого экземпляра 4 признака.



Теперь давайте построим модель нейросети

Сначала попробуем простейшую архитектуру:
Линейный слой, софтмакс

In [19]:
#Линейный слой - это матрица, на которую мы умножаем вектор, который получился на
#предыдущем слое по правилам матричного умножения. Это значит, что кол-во строк
#матрицы весов слоя должно быть равно размеру входного вектора
#А кол-во столбцов - размеру следующего слоя, то есть 3 в нашем случае, так как
#первый слой является последним, то есть выдаёт три числа - показатели трёх классов


log_softmax = torch.nn.LogSoftmax(dim=1)
from collections import OrderedDict

model = torch.nn.Sequential(
    OrderedDict([
        ("linear_1", torch.nn.Linear(4,3)),
        ("log_softmax", torch.nn.LogSoftmax(dim=1)),
    ])
)

teaching = torch.optim.SGD(model.parameters(), lr = 1.)
loss_fn = torch.nn.NLLLoss()
#обучим модель циклом, размер батча будет равен числу экземпляров датасета
for step in range(1, 30001):
    teaching.zero_grad()
    y_pred = model(train_data)
    loss_fn(y_pred, target_data).backward()
    teaching.step()



In [23]:
print((model(train_data).argmax(dim=1)==target_data).sum()/train_data.size(dim=0)) #ссчитаем accuracy

tensor(0.9867)
